In [28]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [29]:
pd.options.display.max_columns = 31

In [30]:
offers = pd.read_csv("offer_acceptance_offers.csv",low_memory=False)
orders = pd.read_csv('offer_acceptance_orders.csv',low_memory=False)

In [31]:
# variables used for modeling
bool_column_names = [
 'FD_ENABLED',
 'EXCLUSIVE_USE_REQUESTED',
 'HAZARDOUS',
 'REEFER_ALLOWED',
 'STRAIGHT_TRUCK_ALLOWED',
 'LOAD_TO_RIDE_REQUESTED',
]

numerical_loggable_column_names = [
 'APPROXIMATE_DRIVING_ROUTE_MILEAGE',
 'PALLETIZED_LINEAR_FEET',
 'SECONDS_BETWEEN_ORDER_AND_DEADLINE',
 'LOAD_BAR_COUNT',
 'ESTIMATED_COST_AT_ORDER'
]

# Data Cleaning

In [32]:
# dates -> DT & creating new time delta variable
orders['ORDER_DATETIME_PST'] = pd.to_datetime(orders['ORDER_DATETIME_PST'])
orders['PICKUP_DEADLINE_PST'] = pd.to_datetime(orders['PICKUP_DEADLINE_PST'])
offers['CREATED_ON_HQ'] = pd.to_datetime(offers['CREATED_ON_HQ'])
orders['Time_between_Order_pickup'] = (orders['PICKUP_DEADLINE_PST'] - 
                                                orders['ORDER_DATETIME_PST'])

In [33]:
# fill driving distance with mean of in zip code results, most na were in zip code
same = orders[orders['ORIGIN_3DIGIT_ZIP'] ==  orders['DESTINATION_3DIGIT_ZIP']]
mean_same_distance = same['APPROXIMATE_DRIVING_ROUTE_MILEAGE'].dropna().mean()
orders['APPROXIMATE_DRIVING_ROUTE_MILEAGE'] = orders['APPROXIMATE_DRIVING_ROUTE_MILEAGE'].fillna(mean_same_distance)

# fill FD enabled with false
orders['FD_ENABLED'] = orders['FD_ENABLED'].fillna(False)

# drop everything else, cause all had consistance NA 7
# 18 in Transportmode
# change all boolean data back to boolean
containsNa = (orders.isnull().sum() > 1).to_dict()
orders = orders.dropna()
for key,value in containsNa.items():
    if value:
        if orders[key].unique().sum() == 1:
            orders[key] = orders[key].astype(bool)

In [34]:
s = offers.groupby('REFERENCE_NUMBER').count()['CARRIER_ID']
nOffers_rec = s[s < 15]

In [35]:
ftl = orders[orders['TRANSPORT_MODE'] == 'FTL']
ftl.set_index('REFERENCE_NUMBER',inplace = True)
joinedDF = ftl.join(nOffers_rec,how = 'left')
joinedDF['NUMBER_OFFERS'] = joinedDF['CARRIER_ID'].fillna(0)
joinedDF.drop('CARRIER_ID',axis = 1,inplace = True)
joinedDF['SECONDS_BETWEEN_ORDER_AND_DEADLINE'] = joinedDF['Time_between_Order_pickup'].dt.total_seconds()

In [45]:
accept_offers = offers[offers['LOAD_DELIVERED_FROM_OFFER'] == True]

In [51]:
joined_oo = pd.merge(accept_offers, orders, how='inner')

In [53]:
joined_oo['LEAD_TIME'] =  joined_oo['PICKUP_DEADLINE_PST'] - joined_oo['CREATED_ON_HQ']

In [56]:
#joined table of both ftl/ptl offers that are accepeted, column LEAD_TIME is time between accpt offr and pickupdeadline
joined_oo['LEAD_TIME'].describe()

count                       126411
mean     1 days 11:55:15.270799218
std      3 days 02:23:02.503108674
min            -149 days +19:24:32
25%         0 days 05:33:09.500000
50%                0 days 21:38:57
75%         1 days 19:42:24.500000
max              365 days 07:07:41
Name: LEAD_TIME, dtype: object

In [66]:
def bin_leadtime(df):
    if df['LEAD_TIME'] >= (df.quantile(q=0.75)['LEAD_TIME']).astype('dateime'):
        return 4
    elif df['LEAD_TIME'] >= (df.quantile(q=0.5)['LEAD_TIME']).astype('dateime'):
        return 3
    elif df['LEAD_TIME'] >= (df.quantile(q=0.25)['LEAD_TIME']).astype('dateime'):
        return 2
    return 1

In [67]:
joined_oo['LEADTIME_BIN'] = joined_oo.apply(bin_leadtime, axis=1)

TypeError: '<' not supported between instances of 'Timestamp' and 'str'

In [71]:
joined_oo.quantile(q=0.75)

RATE_USD                             3870.0000
SELF_SERVE                              0.0000
IS_OFFER_APPROVED                       1.0000
AUTOMATICALLY_APPROVED                  0.0000
MANUALLY_APPROVED                       1.0000
WAS_EVER_UNCOVERED                      0.0000
COVERING_OFFER                          1.0000
LOAD_DELIVERED_FROM_OFFER               1.0000
RECOMMENDED_LOAD                        0.0000
VALID                                   1.0000
APPROXIMATE_DRIVING_ROUTE_MILEAGE    1328.0000
PALLETIZED_LINEAR_FEET                 52.0000
FD_ENABLED                              1.0000
EXCLUSIVE_USE_REQUESTED                 0.0000
HAZARDOUS                               0.0000
REEFER_ALLOWED                          1.0000
STRAIGHT_TRUCK_ALLOWED                  1.0000
LOAD_BAR_COUNT                          0.0000
LOAD_TO_RIDE_REQUESTED                  1.0000
ESTIMATED_COST_AT_ORDER              3741.9288
Name: 0.75, dtype: float64

In [36]:
train=joinedDF.sample(frac=0.8,random_state=200)
test=joinedDF.drop(train.index)

In [37]:
bool_column_names + numerical_loggable_column_names

['FD_ENABLED',
 'EXCLUSIVE_USE_REQUESTED',
 'HAZARDOUS',
 'REEFER_ALLOWED',
 'STRAIGHT_TRUCK_ALLOWED',
 'LOAD_TO_RIDE_REQUESTED',
 'APPROXIMATE_DRIVING_ROUTE_MILEAGE',
 'PALLETIZED_LINEAR_FEET',
 'SECONDS_BETWEEN_ORDER_AND_DEADLINE',
 'LOAD_BAR_COUNT',
 'ESTIMATED_COST_AT_ORDER']

In [38]:
xTrain = train[bool_column_names + numerical_loggable_column_names].to_numpy()
yTrain = train['NUMBER_OFFERS'].to_numpy()
xTest = test[bool_column_names + numerical_loggable_column_names].to_numpy()
yTest = test['NUMBER_OFFERS'].to_numpy()

In [12]:
reg = LinearRegression().fit(xTrain, yTrain)
preds = reg.predict(xTest).round()
mean_absolute_error(yTest,preds)

2.1593948144892963

In [13]:
pd.Series(preds).value_counts()

3.0    11115
4.0     2768
2.0     1923
1.0       85
5.0       38
dtype: int64

In [14]:
avg_pred = [np.mean(nOffers_rec)] * len(yTest)
mean_absolute_error(yTest,avg_pred)

2.3643080876431477

In [15]:
np.bincount(yTest.astype(int))

array([2455, 3510, 2797, 2043, 1446, 1029,  734,  541,  367,  274,  219,
        179,  147,  105,   83])

In [16]:
np.bincount(preds.astype(int))

array([    0,    85,  1923, 11115,  2768,    38])

In [17]:
np.unique(preds.astype(int))

array([1, 2, 3, 4, 5])

In [18]:
offers.index

RangeIndex(start=0, stop=707418, step=1)

In [19]:
joinedTable = test.join(offers.set_index('REFERENCE_NUMBER'),how = 'inner')

In [20]:
joinedTable

,ORDER_DATETIME_PST,PICKUP_DEADLINE_PST,DELIVERY_TIME_CONSTRAINT,ORIGIN_3DIGIT_ZIP,DESTINATION_3DIGIT_ZIP,APPROXIMATE_DRIVING_ROUTE_MILEAGE,PALLETIZED_LINEAR_FEET,FD_ENABLED,EXCLUSIVE_USE_REQUESTED,HAZARDOUS,REEFER_ALLOWED,STRAIGHT_TRUCK_ALLOWED,LOAD_BAR_COUNT,LOAD_TO_RIDE_REQUESTED,ESTIMATED_COST_AT_ORDER,...,NUMBER_OFFERS,SECONDS_BETWEEN_ORDER_AND_DEADLINE,CARRIER_ID,CREATED_ON_HQ,RATE_USD,OFFER_TYPE,SELF_SERVE,IS_OFFER_APPROVED,AUTOMATICALLY_APPROVED,MANUALLY_APPROVED,WAS_EVER_UNCOVERED,COVERING_OFFER,LOAD_DELIVERED_FROM_OFFER,RECOMMENDED_LOAD,VALID
REFERENCE_NUMBER,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"[\n ""0003d127b27fd50722f3e43a6e3b19562ed5a176d87a5e49200bbb8a14643371""\n]",2022-06-16 12:44:16,2022-06-24 13:00:00,SCHEDULE,281,750,1061.0,22.0,True,True,False,False,False,0.0,True,3024.8049,...,3.0,692144.0,d5b925f7508544b836a758dfd0c5a6fb316694cee8be82...,2022-06-20 11:48:15,3096.0,quote,False,True,False,True,False,True,True,False,True
"[\n ""0003d127b27fd50722f3e43a6e3b19562ed5a176d87a5e49200bbb8a14643371""\n]",2022-06-16 12:44:16,2022-06-24 13:00:00,SCHEDULE,281,750,1061.0,22.0,True,True,False,False,False,0.0,True,3024.8049,...,3.0,692144.0,ccf51d90eb2019002bd4b6f104a5c4dec9aa4fc2fb3672...,2022-06-20 07:59:23,3096.0,quote,False,False,False,False,False,False,False,False,False
"[\n ""0003d127b27fd50722f3e43a6e3b19562ed5a176d87a5e49200bbb8a14643371""\n]",2022-06-16 12:44:16,2022-06-24 13:00:00,SCHEDULE,281,750,1061.0,22.0,True,True,False,False,False,0.0,True,3024.8049,...,3.0,692144.0,d5b925f7508544b836a758dfd0c5a6fb316694cee8be82...,2022-06-20 07:59:38,3096.0,quote,False,False,False,False,False,False,False,False,True
"[\n ""0004b681b1c4f14dc289a36cf0590ffce5b6aa2220f6b340d550c952ea4caf23""\n]",2020-08-17 15:29:20,2020-08-18 15:30:00,NONE,900,276,2547.0,24.0,True,True,False,True,True,0.0,False,9245.7654,...,1.0,86440.0,c25ed90a126cbdabb004b777139973b0652f84051907c8...,2020-08-17 15:47:22,9288.0,quote,False,False,False,False,False,True,True,False,True
"[\n ""000880c3b32626141c08b5040c79d5bdb72af4225f06a1f1615c686f97e9775f""\n]",2022-02-24 09:07:45,2022-02-28 14:00:00,NONE,716,515,611.0,52.0,True,True,False,False,False,0.0,False,2956.8993,...,1.0,363135.0,62fcdb6ab7d866469212eb68cea38fd06fa6c92dc597ad...,2022-02-25 12:03:57,2709.0,quote,False,True,True,False,False,True,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"[\n ""fff6f947ed25527c7ac3e4c2e598bbfaa17fa4f60e8bea8a64c69b216424da53""\n]",2022-08-22 07:52:21,2022-08-30 11:00:00,SCHEDULE,305,720,606.0,53.0,True,False,False,True,True,0.0,False,1829.2716,...,4.0,702459.0,a4f3530830cabf2d9e40d2971ea6a17e38535e764bccc3...,2022-08-26 08:38:59,2128.5,quote,True,False,False,False,False,False,False,True,True
"[\n ""fffc15629ba0aa26f5252d1b619e01cf10306f97b440027ee7dc899a4f59b107""\n]",2021-03-29 15:39:19,2021-03-31 11:00:00,SCHEDULE,303,945,2471.0,47.0,True,True,False,True,False,0.0,True,6936.4461,...,4.0,156041.0,742a443c030dc6f372319158da396c1005b2930fcc943e...,2021-03-30 09:17:41,6450.0,quote,False,True,False,True,False,True,True,False,True
"[\n ""fffc15629ba0aa26f5252d1b619e01cf10306f97b440027ee7dc899a4f59b107""\n]",2021-03-29 15:39:19,2021-03-31 11:00:00,SCHEDULE,303,945,2471.0,47.0,True,True,False,True,False,0.0,True,6936.4461,...,4.0,156041.0,7ca2552ffd0e38deee9b22af6d3e8ce9fe88ec88162469...,2021-03-30 07:35:52,6708.0,quote,False,False,False,False,False,False,False,False,True


In [21]:
uniqueRefs = joinedTable.index.unique()[:500]

In [22]:
#joinedTable['OFF_TO_ACCEPT'] = joined

In [25]:
test = test.reset_index()

In [26]:
accepted_offer = []
for i in uniqueRefs:
    best_calc = int(test.where(test['REFERENCE_NUMBER'] == i).dropna()['Estimated_Num_offers']/np.e)
    offers_at_ref = list(offers.where(offers['REFERENCE_NUMBER'] == i).dropna())
    accepted_offer.append((str(i), offers_at_ref[best_calc-1]))
    

KeyError: 'Estimated_Num_offers'

In [27]:
offers_at_ref

NameError: name 'offers_at_ref' is not defined

In [74]:
offers = offers.reset_index().sort_values(by = ['REFERENCE_NUMBER', 'CREATED_ON_HQ'], na_position = 'first')

In [23]:
test['Estimated_Num_offers'] = preds.astype(int)
test

,ORDER_DATETIME_PST,PICKUP_DEADLINE_PST,DELIVERY_TIME_CONSTRAINT,ORIGIN_3DIGIT_ZIP,DESTINATION_3DIGIT_ZIP,APPROXIMATE_DRIVING_ROUTE_MILEAGE,PALLETIZED_LINEAR_FEET,FD_ENABLED,EXCLUSIVE_USE_REQUESTED,HAZARDOUS,REEFER_ALLOWED,STRAIGHT_TRUCK_ALLOWED,LOAD_BAR_COUNT,LOAD_TO_RIDE_REQUESTED,ESTIMATED_COST_AT_ORDER,TRANSPORT_MODE,Time_between_Order_pickup,NUMBER_OFFERS,SECONDS_BETWEEN_ORDER_AND_DEADLINE,Estimated_Num_offers
REFERENCE_NUMBER,,,,,,,,,,,,,,,,,,,,
"[\n ""0003d127b27fd50722f3e43a6e3b19562ed5a176d87a5e49200bbb8a14643371""\n]",2022-06-16 12:44:16,2022-06-24 13:00:00,SCHEDULE,281,750,1061.0,22.0,True,True,False,False,False,0.0,True,3024.8049,FTL,8 days 00:15:44,3.0,692144.0,4
"[\n ""0004b681b1c4f14dc289a36cf0590ffce5b6aa2220f6b340d550c952ea4caf23""\n]",2020-08-17 15:29:20,2020-08-18 15:30:00,NONE,900,276,2547.0,24.0,True,True,False,True,True,0.0,False,9245.7654,FTL,1 days 00:00:40,1.0,86440.0,3
"[\n ""000880c3b32626141c08b5040c79d5bdb72af4225f06a1f1615c686f97e9775f""\n]",2022-02-24 09:07:45,2022-02-28 14:00:00,NONE,716,515,611.0,52.0,True,True,False,False,False,0.0,False,2956.8993,FTL,4 days 04:52:15,1.0,363135.0,3
"[\n ""000d3a9bad1e2fa2c921b7565f21f5d31fe9291f02c9d5fb4495825fcfeb2dc3""\n]",2022-08-22 08:36:55,2022-09-07 13:00:00,SCHEDULE,305,704,503.0,52.0,True,False,False,True,True,0.0,False,1641.6411,FTL,16 days 04:23:05,0.0,1398185.0,3
"[\n ""0011ec19bd38340c2a86d26fee984485ed966a901ffc8bad78c00ba1a4a32fc8""\n]",2021-09-30 06:29:59,2021-10-05 07:00:00,SCHEDULE,296,281,139.0,46.0,True,False,False,True,True,0.0,False,896.8596,FTL,5 days 00:30:01,2.0,433801.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"[\n ""ffe54c86ce7ef7be88217ab660f339c45e46e1292dca424bb012130d100c30ea""\n]",2021-11-22 14:02:31,2021-11-29 13:00:00,SCHEDULE,303,282,256.0,100.0,True,True,False,True,True,0.0,True,1411.5309,FTL,6 days 22:57:29,0.0,601049.0,2
"[\n ""ffef1a34842ba031f5fbf07203651a9116a447edf08cc32453a36eac8d783296""\n]",2022-09-16 13:55:55,2022-09-20 15:00:00,SCHEDULE,841,604,1397.0,100.0,True,True,False,False,False,0.0,True,3099.6636,FTL,4 days 01:04:05,0.0,349445.0,3
"[\n ""fff53791fdea029938fbbc7f613aaf68505f9537129d2d67c5e48a20716a0d4a""\n]",2022-08-15 10:50:30,2022-08-16 12:00:00,SCHEDULE,303,296,166.0,48.0,True,True,False,False,False,0.0,True,965.5263,FTL,1 days 01:09:30,3.0,90570.0,3


# FTL

In [24]:
ftl = orders[orders['TRANSPORT_MODE'] == 'FTL']
ftl.set_index('REFERENCE_NUMBER',inplace = True)
offers.set_index('REFERENCE_NUMBER',inplace = True)

In [25]:
joinedDF = ftl.join(offers,how = 'left')
joinedDF.head()

,ORDER_DATETIME_PST,PICKUP_DEADLINE_PST,DELIVERY_TIME_CONSTRAINT,ORIGIN_3DIGIT_ZIP,DESTINATION_3DIGIT_ZIP,APPROXIMATE_DRIVING_ROUTE_MILEAGE,PALLETIZED_LINEAR_FEET,FD_ENABLED,EXCLUSIVE_USE_REQUESTED,HAZARDOUS,REEFER_ALLOWED,STRAIGHT_TRUCK_ALLOWED,LOAD_BAR_COUNT,LOAD_TO_RIDE_REQUESTED,ESTIMATED_COST_AT_ORDER,TRANSPORT_MODE,Time_between_Order_pickup,CARRIER_ID,CREATED_ON_HQ,RATE_USD,OFFER_TYPE,SELF_SERVE,IS_OFFER_APPROVED,AUTOMATICALLY_APPROVED,MANUALLY_APPROVED,WAS_EVER_UNCOVERED,COVERING_OFFER,LOAD_DELIVERED_FROM_OFFER,RECOMMENDED_LOAD,VALID
REFERENCE_NUMBER,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"[\n ""000020adb97f32cf89e5f094b35607499f7ff872ff0045411bca69345c97c70c""\n]",2022-08-18 12:45:34,2022-08-24 11:00:00,SCHEDULE,761,304,1025.0,52.0,True,True,False,False,False,0.0,True,2353.605,FTL,5 days 22:14:26,23cdd018d515d5e59464aa65d8eb5b781e3a4872e18aa2...,2022-08-24 07:12:40,2967.0,quote,False,True,False,True,False,True,True,False,True
"[\n ""000020adb97f32cf89e5f094b35607499f7ff872ff0045411bca69345c97c70c""\n]",2022-08-18 12:45:34,2022-08-24 11:00:00,SCHEDULE,761,304,1025.0,52.0,True,True,False,False,False,0.0,True,2353.605,FTL,5 days 22:14:26,f91b043302878951ce9258214033bd206ea0a92bb88931...,2022-08-23 10:55:30,3483.0,quote,False,False,False,False,False,False,False,False,False
"[\n ""000020adb97f32cf89e5f094b35607499f7ff872ff0045411bca69345c97c70c""\n]",2022-08-18 12:45:34,2022-08-24 11:00:00,SCHEDULE,761,304,1025.0,52.0,True,True,False,False,False,0.0,True,2353.605,FTL,5 days 22:14:26,6326cfe26285d0f687c2cc5385aad8a0f7b2664a30c84a...,2022-08-23 10:18:47,3483.0,quote,False,False,False,False,False,False,False,False,True
"[\n ""000020adb97f32cf89e5f094b35607499f7ff872ff0045411bca69345c97c70c""\n]",2022-08-18 12:45:34,2022-08-24 11:00:00,SCHEDULE,761,304,1025.0,52.0,True,True,False,False,False,0.0,True,2353.605,FTL,5 days 22:14:26,2fe98face8187bcd9a099889cbb3d02c4a80b9fc72ec9f...,2022-08-24 05:14:46,3225.0,quote,True,False,False,False,False,False,False,True,True
"[\n ""000020adb97f32cf89e5f094b35607499f7ff872ff0045411bca69345c97c70c""\n]",2022-08-18 12:45:34,2022-08-24 11:00:00,SCHEDULE,761,304,1025.0,52.0,True,True,False,False,False,0.0,True,2353.605,FTL,5 days 22:14:26,f91b043302878951ce9258214033bd206ea0a92bb88931...,2022-08-23 10:57:27,3483.0,quote,True,False,False,False,False,False,False,False,True


## Do Orders get 0 offers

In [26]:
off = offers.index.to_list()
z = joinedDF[joinedDF['VALID'].isnull()].index.to_list()[:100]
orde =orders['REFERENCE_NUMBER'].to_list()
numOff = 0
numOrd = 0
for i in z:
    if i in off:
        numOff +=1
    if i in orde:
        numOrd +=1
numOff,numOrd

(0, 100)

### yes orders get 0 offers, about 10919 orders without offer. 

In [27]:
# create new column whether order recieved offer
joinedDF['Recieved_Offer'] = joinedDF.notnull()['VALID']
joinedDF['Recieved_Offer']

REFERENCE_NUMBER
[\n  "000020adb97f32cf89e5f094b35607499f7ff872ff0045411bca69345c97c70c"\n]     True
[\n  "000020adb97f32cf89e5f094b35607499f7ff872ff0045411bca69345c97c70c"\n]     True
[\n  "000020adb97f32cf89e5f094b35607499f7ff872ff0045411bca69345c97c70c"\n]     True
[\n  "000020adb97f32cf89e5f094b35607499f7ff872ff0045411bca69345c97c70c"\n]     True
[\n  "000020adb97f32cf89e5f094b35607499f7ff872ff0045411bca69345c97c70c"\n]     True
                                                                              ...  
[\n  "fffed2af60b0061cbe9423f48735baff489a073f3a7bb46606c0cf623bf7d5bc"\n]    False
[\n  "ffff4267ee0ce72d72a1ca2cf36af423d7d61405aac57a4b03df8b7ae4dba3ee"\n]     True
[\n  "ffff4267ee0ce72d72a1ca2cf36af423d7d61405aac57a4b03df8b7ae4dba3ee"\n]     True
[\n  "ffff4267ee0ce72d72a1ca2cf36af423d7d61405aac57a4b03df8b7ae4dba3ee"\n]     True
[\n  "ffff4267ee0ce72d72a1ca2cf36af423d7d61405aac57a4b03df8b7ae4dba3ee"\n]     True
Name: Recieved_Offer, Length: 280273, dtype: bool

# Results of Flock Method vs Best possible

In [28]:
# removing Uncovered
orderWoffer = joinedDF[joinedDF['Recieved_Offer']]
orderWoffer.reset_index(inplace = True)
cover = orderWoffer.drop(orderWoffer[orderWoffer['WAS_EVER_UNCOVERED']].index)

In [29]:
indexIn = cover.set_index('REFERENCE_NUMBER')
flockAccept = indexIn[indexIn['LOAD_DELIVERED_FROM_OFFER']].RATE_USD
flockN = len(flockAccept)
flockT = sum(flockAccept)
flockM = np.mean(flockAccept)
print('Number of Orders Fulfilled: ' + str(flockN))
print('Total Contract Value: ' + str(flockT))
print('Mean Contract Value: ' + str(flockM))

Number of Orders Fulfilled: 62103
Total Contract Value: 209625472.59150046
Mean Contract Value: 3375.448409762821


In [30]:
bestAccept = cover.groupby('REFERENCE_NUMBER')['RATE_USD'].min()
bestN = len(bestAccept)
bestT = sum(bestAccept)
bestM = np.mean(bestAccept)

print('Number of Orders Fulfilled: ' + str(bestN))
print('Total Contract Value: ' + str(bestT))
print('Mean Contract Value: ' + str(bestM))

Number of Orders Fulfilled: 68222
Total Contract Value: 226338405.93270046
Mean Contract Value: 3317.674737367718


In [31]:
print('Flock Missed: ' + str(bestN-flockN))
print('Extra Spent: ' + str(bestT-flockT))
print('Mean Contract Value: ' + str(bestM-flockM))

Flock Missed: 6119
Extra Spent: 16712933.341199994
Mean Contract Value: -57.77367239510295


In [32]:
# how flock could have done with offer
tf = cover.groupby('REFERENCE_NUMBER')['LOAD_DELIVERED_FROM_OFFER'].apply(any)
pot = bestAccept[tf]
bestN = len(pot)
bestT = sum(pot)
bestM = np.mean(pot)

print('Number of Orders Fulfilled: ' + str(bestN))
print('Total Contract Value: ' + str(bestT))
print('Mean Contract Value: ' + str(bestM))

Number of Orders Fulfilled: 62103
Total Contract Value: 204460973.00700065
Mean Contract Value: 3292.2881826481917


In [33]:
print('Flock Missed: ' + str(bestN-flockN))
print('Extra Spent: ' + str(bestT-flockT))
print('Mean Contract Value: ' + str(bestM-flockM))

Flock Missed: 0
Extra Spent: -5164499.584499806
Mean Contract Value: -83.16022711462938


# Summary of Analysis FTL

#### Flock missed out on 6119 contracts by not accepting anything
#### Flock could have saved 58 dollars on average if getting the best offer everytime while doing the extra 6119
#### Flock could have saved 83 dollars on average if getting the best offer on the ones they offered
#### Flock Picked 12,413 wrong and 7 cost them over 10,000 dollars each

# How well did the estimate offer do

In [34]:
estimate = cover.groupby('REFERENCE_NUMBER')['ESTIMATED_COST_AT_ORDER'].mean()

In [35]:
estimate

REFERENCE_NUMBER
[\n  "000020adb97f32cf89e5f094b35607499f7ff872ff0045411bca69345c97c70c"\n]    2353.6050
[\n  "0002e80902489c5a5eed029971f32e2ade3513870d804c653e8fcb3523a431aa"\n]    5243.3082
[\n  "0003d127b27fd50722f3e43a6e3b19562ed5a176d87a5e49200bbb8a14643371"\n]    3024.8049
[\n  "0004b681b1c4f14dc289a36cf0590ffce5b6aa2220f6b340d550c952ea4caf23"\n]    9245.7654
[\n  "0004ef769b6e6e2a753c030ba14bfb33c88e392cf70a3acbdd840a5bda2f4fd0"\n]     835.7394
                                                                                ...    
[\n  "fffb240d62d732bd179440f15272283f408368ed1595415e5498d3457eafc659"\n]    7384.5534
[\n  "fffbec3e1ffcf0c505877a2b68e8fc773a413f9db25cc40fb66eff17b333354a"\n]    9373.3464
[\n  "fffc15629ba0aa26f5252d1b619e01cf10306f97b440027ee7dc899a4f59b107"\n]    6936.4461
[\n  "fffdb6cd77e2d5e8bfa5a355cf80a2c90a1a800f086add7b12f7d0ae1e2ecdca"\n]    2848.8618
[\n  "ffff4267ee0ce72d72a1ca2cf36af423d7d61405aac57a4b03df8b7ae4dba3ee"\n]    2565.2682
Name: ESTIMATED

In [36]:
bestAccept

REFERENCE_NUMBER
[\n  "000020adb97f32cf89e5f094b35607499f7ff872ff0045411bca69345c97c70c"\n]    2967.0
[\n  "0002e80902489c5a5eed029971f32e2ade3513870d804c653e8fcb3523a431aa"\n]    5160.0
[\n  "0003d127b27fd50722f3e43a6e3b19562ed5a176d87a5e49200bbb8a14643371"\n]    3096.0
[\n  "0004b681b1c4f14dc289a36cf0590ffce5b6aa2220f6b340d550c952ea4caf23"\n]    9288.0
[\n  "0004ef769b6e6e2a753c030ba14bfb33c88e392cf70a3acbdd840a5bda2f4fd0"\n]     774.0
                                                                               ...  
[\n  "fffb240d62d732bd179440f15272283f408368ed1595415e5498d3457eafc659"\n]    7740.0
[\n  "fffbec3e1ffcf0c505877a2b68e8fc773a413f9db25cc40fb66eff17b333354a"\n]    9417.0
[\n  "fffc15629ba0aa26f5252d1b619e01cf10306f97b440027ee7dc899a4f59b107"\n]    6450.0
[\n  "fffdb6cd77e2d5e8bfa5a355cf80a2c90a1a800f086add7b12f7d0ae1e2ecdca"\n]    2838.0
[\n  "ffff4267ee0ce72d72a1ca2cf36af423d7d61405aac57a4b03df8b7ae4dba3ee"\n]    2451.0
Name: RATE_USD, Length: 68222, dtype: float64

In [37]:
frame = { 'Estimate': estimate, 'Accepted': flockAccept,'Best': bestAccept}
#Creating DataFrame by passing Dictionary
result = pd.DataFrame(frame)

In [38]:
result['estVSbest'] = np.abs(result['Estimate'] - result['Best'])
result['estVSreal'] = np.abs(result['Estimate'] - result['Accepted'])
np.mean(result)

/opt/conda/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3370: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


Estimate     3388.905234
Accepted     3375.448410
Best         3317.674737
estVSbest     331.680834
estVSreal     334.940814
dtype: float64

In [39]:
result.mean()

Estimate     3388.905234
Accepted     3375.448410
Best         3317.674737
estVSbest     331.680834
estVSreal     334.940814
dtype: float64

In [40]:
sum(result['estVSreal'] > 0)

61867

In [41]:
sum(result['estVSreal'] < 0)

0

# PTL

In [42]:
ptl = orders[orders['TRANSPORT_MODE'] == 'PTL']
ptl.groupby('REFERENCE_NUMBER').count()['ORDER_DATETIME_PST'].sort_values()

REFERENCE_NUMBER
[\n  "00003e8484fe610f9a8040b7ab5cc4593c50186a1efd30d61cb6a49109484486"\n]    1
[\n  "aa0a913da9877658e4ec3a932facccb51d3fd29841d51d9bd2392e0eaeeb2271"\n]    1
[\n  "aa0a4345921cefee0219a66125fe28287ce6c0023610510126442a902d1c2ee5"\n]    1
[\n  "aa0a0be39d7ec1dabc1625ac697a9e2e4985f17515509594a0db458354cf4e21"\n]    1
[\n  "aa096a892fe2c009fdf0374f0564444eac6322809063cf5d41791440ec1a455c"\n]    1
                                                                             ..
[\n  "674033cf6c9aa8fe0550c49c3ea22ace85857cdb71f8f899add66d9752377772"\n]    4
[\n  "de513fbf2547a8a1194f5f3233c902dd73605274aee0aeda673c137ad864457e"\n]    4
[\n  "b12e3ee65eb2bfdec4c36e28da67cde3d61c1d56d2d8a124f147bdd222cd7001"\n]    4
[\n  "66ac2263dd12859e37491bf72efbad9fe696f9cd13b67b261dd1effd0b2686b6"\n]    4
[\n  "7689dfa37fa7621c67e911b0b72d8b2339ef4810647f30685ae8da9e0cbe5793"\n]    4
Name: ORDER_DATETIME_PST, Length: 134724, dtype: int64